In [2]:
import pandas as pd
df = pd.read_csv('../../top_rated_wines.csv')
df = df[df['variety'].notna()] # remove any NaN values as it blows up serialization
data = df.to_dict('records')
df

ModuleNotFoundError: No module named 'pandas'

In [9]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [10]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [11]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [12]:
# Create collection to store books
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

True

In [23]:
# vectorize!
qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [24]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode("99 points Cabernet Sauvignon from Napa Valley").tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': "Anderson's Conn Valley Vineyards Cabernet Sauvignon Estate Reserve 2008", 'region': 'Napa Valley, California', 'variety': 'Red Wine', 'rating': 98.0, 'notes': '100% Cabernet Sauvignon, 100% Estate grown.'} score: 0.7232318357735968
{'name': "Anderson's Conn Valley Vineyards Cabernet Sauvignon Reserve (1.5 Liter Magnum) 2008", 'region': 'Napa Valley, California', 'variety': 'Red Wine', 'rating': 98.0, 'notes': '100% Cabernet Sauvignon, 100% Estate grown.'} score: 0.7232318357735968
{'name': "L'Aventure Estate Cuvee 2014", 'region': 'Central Coast, California', 'variety': 'Red Wine', 'rating': 98.0, 'notes': 'Blend: 50% Cabernet Sauvignon, 35% Syrah, 15% Petit Verdot'} score: 0.6844836784212434
